# Part 1: Pulse Rate Algorithm

## Code description

The code below estimates pulse rates in beats per minutes (BPM) from a photoplethysmogram (PPG) taken from a wrist-wearable device. The code compares the frequencies of the PPG with the ones of a 3-axis accelerometer in order to compensate for the motion component. This is due to the arm movement while walking or running, where another periodic signal appears in the PPG due to this arm motion. Furthermore, the code takes into consideration only pulse rates bewteen 40 and 240 BPM.

The code generates the pulse rate estimates along with a given confidence and error. Ground truth data from electrocardiogram (ECG) signals serve to compare the estimates and give the mean absolute errors.

There are two main functions to run the code:
- `RunPulseRateAlgorithm(data_fl, ref_fl)`: Runs the algorithm for a given sample by passing the sensor measurements file and the ground truth file.
- `Evaluate()`: Runs the algorithm on all measurements and generates an error estimate.

## Data description

The algorithm uses the **Troika**[1] dataset. This dataset has the following features:

- Two-channel PPG signals
- Three-axis ACC signals
- One-channel ECG signals

The measurements correspond to 12 samples of people in the range of 18 to 35 years old, sampled at 125 Hz. Both the PPG and ACC were recorded by a wristband and the ECG from the chest using ECG sensors. The measurements correspond to running activities on a treadmill with changing speeds.

**Limitations:** 
- The dataset does not indicate fitness level, sex, ethnic, and dietary habits of the subjects. Furthremore, each activity has a limited sample duration.
- The samples are taken at short time intervals. There is no constant tracking.

**Reference:**
1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [2]:
import glob

import numpy as np
import scipy as sp
import scipy.io

from matplotlib import pyplot as plt
from scipy import signal as sg


FS = 125  # All signals were sampled at 125 Hz

# The following parameters match the structure of the test data
WINDOW_LENGTH_SEC = 8  # Window length used in to compute the spectogram frequencies
OVERLAP = 6  # Two successive time windows overlap by 6 seconds


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def LoadTroikaRefFile(ref_fl):
    """
    Loads and extracts the ground-truth of heart rate from a troika data file.

    Args:
        ref_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for the BPM value in every 8-second time window. Note that two successive time windows
overlap by 6 seconds. Thus the first value in 'BPM0' gives the calcualted heart rate ground-truth in
the first 8 seconds, while the second value in 'BPM0' gives the calculated heart rate ground-truth
from the 3rd second to the 10th second. 
    """
    return sp.io.loadmat(ref_fl)['BPM0']


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))


def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)


def bpm_to_hz(bpm):
    """
    Convert a magnitude given in Beats per Minute (BPM) to Hertz (Hz)
    """
    return bpm / 60


def hz_to_bpm(hz):
    """
    Convert a magnitude given in Hertz (Hz) to Beats per Minute (BPM)
    """
    return hz * 60


def BandpassFilter(signal):
    """
    A Bandpass filter to remove frequencies outside the range 40-240 Beats per Minute
    
    Returns:
        A ndarray containing the filtered signal 
    """
    low_freq, high_freq = bpm_to_hz(40), bpm_to_hz(240)
    b, a = sg.butter(5, (low_freq, high_freq), btype='bandpass', fs=FS)
    return sg.filtfilt(b, a, signal)


def Spectogram(signal):
    """
    Compute the spectogram and frequencies of a given signal.
    
    Data are split into segments of 8 seconds with an overlap of 6 seconds.
    
    Returns:
        spec: 2D ndarray
            Columns containing the periodograms of successive segments.
        freqs: 1D ndarray
            The frequencies corresponding to the rows in spectrum.
    """
    _ = plt.figure()
    plt.title(f'Spectrogram')
    spec, freqs, _, _ = plt.specgram(
        signal, 
        Fs=FS, 
        NFFT=WINDOW_LENGTH_SEC * FS, 
        noverlap=OVERLAP * FS,
    )
    plt.xlabel('Time (sec)')
    plt.ylabel('Frequency (Hz)')
    plt.close()  # Comment to show plot
    return spec, freqs


def SimilarFreqs(freq_a, window_hz, freq_b):
    """
    Indicates whether the difference between two frequencies fall within a window.
    """
    return (
        (freq_a >= freq_b - window_hz / 2) &
        (freq_a <= freq_b + window_hz / 2)
    )


def EstimationConfidence(spec, hr_freqs, all_freqs):
    """
    Computes the confidence of the estimations
    
    The confidence is given by the energy around the estimated frequency
    over the total energy along the spectrum.
    
    Returns:
        confidences: 1D ndarray
            Array with the energy ratios
    """
    # Get freqs around the heart rate
    w_f = bpm_to_hz(10)
    
    confidences = []
    for i, time_window in enumerate(hr_freqs):
        fund_freqs = (all_freqs >= hr_freqs[i] - w_f) & (all_freqs <= hr_freqs[i] + w_f)    
    
        hr_signal = np.sum(spec[:, i][fund_freqs])
        all_signals = np.sum(spec[:, i])
        
        confidences.append(hr_signal / all_signals)
    return np.array(confidences)


def SimilarToACC(ppg_freq, window, freq_accx, freq_accy, freq_accz):
    """
    Compares a PPG frequency with the 3 axis of the Accelerometer
    
    Returns:
        Boolean indicating whether the frequencies fall into a similar range
    """
    return (
        SimilarFreqs(ppg_freq, window, freq_accx) |
        SimilarFreqs(ppg_freq, window, freq_accy) |
        SimilarFreqs(ppg_freq, window, freq_accz)
    )


def FindHeartbeatFrequency(
    top_ppg_freqs, 
    dom_freq_accx, 
    dom_freq_accy, 
    dom_freq_accz,
    avg_latest_freqs,
):
    """
    Find the heart beat frequency given by the PPG sensor
    
    The algorithm iterates over the top PPG frequencies to find the one related to
    heart beats. Frequencies similar to the dominant frequency of the accelerometer
    are skipped. In case all frequencies are similar to the accelerometer, the 
    frequency that resembles the past estimations is considered.
    
    Returns:
        heartbeat_freq: float
            Value of the estimated heartbeat frequency in Hz
    """
    heartbeat_freq = None
    for top_ppg_freq in top_ppg_freqs:
        bpm_window = 15  # Window of frequencies around the
                         # PPG to compare against the ACC
        window = bpm_to_hz(bpm_window)

        if SimilarToACC(top_ppg_freq, window, dom_freq_accx, dom_freq_accy, dom_freq_accz): 
            continue

        heartbeat_freq = top_ppg_freq
        break

    # If the cadence of the arm swing is the same as the heartbeat, take
    # the most similar frequency to the latest samples
    if not heartbeat_freq:
        if avg_latest_freqs > 0:
            diffs = np.abs(top_ppg_freqs - avg_latest_freqs)
            heartbeat_freq = top_ppg_freqs[np.argmin(diffs)]
        else:
            heartbeat_freq = top_ppg_freqs[0]
    return heartbeat_freq


def GetTopKFreqs(k, specs, freqs, time_window):
    """
    Iterates over the frequency spectrum and return the top K frequencies
    that maximise the spectrum value.
    
    Return:
        top_k_freqs: 1D ndarray
            Array of top frequencies sorted in decreasing order
    """
    top_k_freq_idx_asc = np.argpartition(specs[:, time_window], -k)[-k:]
    return freqs[top_k_freq_idx_asc[::-1]]
    

def FindFrequenciesOverTime(ppg, accx, accy, accz):
    """
    Computes the estimated heart beat frequencies through time.
    
    Returns:
        motion_compensated_freqs: 1D ndarray
            Estimated heart beat frequencies in Hz
        spec_ppg: 2D ndarray
            Columns containing the PPG spectrum of successive segments.
        freqs_ppg: 1D ndarray
            The frequencies corresponding to the rows in PPG spectrum.
    """
    signals = (ppg, accx, accy, accz)
    ppg, accx, accy, accz = (BandpassFilter(sig) for sig in signals)
    spec_ppg, freqs_ppg = Spectogram(ppg)
    spec_accx, freqs_accx = Spectogram(accx)
    spec_accy, freqs_accy = Spectogram(accy)
    spec_accz, freqs_accz = Spectogram(accz)
        
    # Find dominant frequencies for every time window
    motion_compensated_freqs = []
    for time_window in range(spec_ppg.shape[1]):        
        top_5_dom_freq_ppg = GetTopKFreqs(5, spec_ppg, freqs_ppg, time_window)

        dom_freq_accx = freqs_accx[np.argmax(spec_accx[:, time_window])]
        dom_freq_accy = freqs_accy[np.argmax(spec_accy[:, time_window])]
        dom_freq_accz = freqs_accz[np.argmax(spec_accz[:, time_window])]
    
        latest_freqs = np.mean(motion_compensated_freqs[-2:]) if motion_compensated_freqs else 0
    
        heartbeat_freq = FindHeartbeatFrequency(
            top_5_dom_freq_ppg,
            dom_freq_accx,
            dom_freq_accy,
            dom_freq_accz,
            latest_freqs
        )
        motion_compensated_freqs.append(heartbeat_freq)
    return np.array(motion_compensated_freqs), spec_ppg, freqs_ppg


def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Estimate pulse rate from the PPG sensor and compare it with ground truth data from an ECG.
    
    Returns:
        errors: 1D ndarray
            Contains the absolute error of the estimations
        confidence: 1D ndarray
            Contains the estimations confidence
    """
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    y_true = LoadTroikaRefFile(ref_fl).flatten()
    
    estimated_freqs, spec_ppg, freqs_ppg = FindFrequenciesOverTime(ppg, accx, accy, accz)    
    confidence = EstimationConfidence(spec_ppg, estimated_freqs, freqs_ppg)

    y_pred = hz_to_bpm(estimated_freqs)
    errors = np.abs(y_true - y_pred)
    return errors, confidence

In [3]:
Evaluate()

12.679878804887686

In [4]:
# Retrieve dataset files
data_fls, ref_fls = LoadTroikaDataset()
len(data_fls)

12

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

## Algorithm description
### How the algoirhtm works

Once the PPG and 3-axis ACC measurements are loaded, the algorithm finds the dominant frequencies of the PPG in windows of 8 seconds, with an overlap of 6 seconds. That is, it produces estimates every 2 seconds. The process of finding the dominant frequencies is where the top 5 PPG dominant frequencies are compared with the signals from the ACC. Starting with the frequency with the highest energy, if it is similar to dominant frequencies of the ACC (in any of the 3 axes), then it is skipped and the next dominant frequency is evaluated. If all the top frequencies of the PPG are similar to the ACC, then the frequency that resembles the previous two estimates is used.

All signals pass by a Bandpass filter that keeps frequencies between 40 and 240 BPMs.

### Relevant physiology aspects
The PPG signal was recorded from the wrist by two pulse oximeters with green LEDs (wavelength: 515nm). The PPG then measures the difference between the light emmited and reflected to estimate the pulse rate. This can be done since every time the ventricles contract, the capillaries in the wrist fill with blood. Then, the reflection of green light indicates how much blood there is in the capillaries. The periodic change in blood flow in these capillaries, are captured by the photodetector and gives an oscillating waveform with the estimated pulse rate.

### Algorithm outputs
The algorithm returns the confidence and estimation errors. The confidence is computed by taking the ration between the sum of the energy around the pulse rate estimate (with a window of 10 BPM) over the sum of the energy over the whole spectrum. The error is given by the absolute difference of the estimate and the ground truth signal in BPM.

### Caveats on algorithm outputs
There are several noise sources that are might influence the algorithm outputs:

* Melanin: There exists different light absortion rates depending on the level of melanin. That results in different levels of confidence according to the skin color.

* Arm motion: As the PPG signal measures pulse rate from blood flow, and this is a liquid, its flow is also influenced by the arm movement and not just by the heart's ventricles contraction.

* Arm position: Similar to the previous point, the arm position determines how the blood flows. Thus, it influences how much light is reflected to the photodetector.

* Finger movement: Finger movements generates movements in the arm tendons, which also might disturb the detection of light reflection. The reasons can be attributed to change in blood flow and a shift in the position of the sensor.


### Common failure modes
The periodic signals detected by the accelerometer come from sensing arm motion. It is unlikely that they are due to the pulse rate. However, due to arm motion, sometimes its frequency is the same as the heart rate. This behaviour makes it difficult for the algorithm to differentiate the origin of the frequency. When it happens, the algorithm chooses the most similar frequency to the latest estimations. By doing so, the algorithm avoids peaks in the pulse estimates when there is a considerable uncertainty in the estimation.

## Algorithm performance
The algorithm achives a mean absolute error (MAE) of 12 BPM, at a 90% availability, on the training dataset.

The test detailed below, based on the Troika test set, the algorithm achieves a MAE of 9.82 BPM at 90% availability, meeting the project requirements.

![test_result](./passed.png)